In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
import re
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Ineuron/NLP/IMDB_Movie_reviews/IMDB Dataset.csv')

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
corpus = []
for i in range(len(data)):
  review = re.sub('[^A-Za-z0-9]',' ',data['review'][i])
  review = review.lower()
  review = review.split()
  review = [stemmer.stem(words) for words in review if words not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [8]:
max_length = max(len(sentence) for sentence in corpus)
max_length

8380

In [9]:
voc_size = 20000

In [10]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]

In [11]:
onehot_repr

[[881,
  15954,
  10235,
  17260,
  14439,
  5846,
  6818,
  16889,
  8363,
  15641,
  5966,
  6239,
  6239,
  8148,
  1443,
  3351,
  5846,
  15415,
  12531,
  3076,
  14059,
  13618,
  8363,
  17793,
  15667,
  15308,
  3598,
  10635,
  4614,
  8763,
  3598,
  15276,
  15772,
  6244,
  13018,
  12758,
  14059,
  3539,
  14822,
  10593,
  17793,
  6239,
  6239,
  12836,
  5846,
  9041,
  5164,
  5024,
  71,
  9380,
  10490,
  18572,
  15939,
  7488,
  4138,
  1097,
  13823,
  4383,
  15296,
  13842,
  4259,
  5171,
  19049,
  18129,
  1583,
  12642,
  4822,
  3571,
  1097,
  2179,
  1460,
  12840,
  12370,
  4686,
  7946,
  16605,
  6101,
  16415,
  7493,
  17478,
  2426,
  4113,
  11671,
  15114,
  9988,
  6304,
  8348,
  15077,
  6239,
  6239,
  16905,
  14353,
  5091,
  6607,
  3598,
  4329,
  6541,
  8731,
  3598,
  19739,
  2205,
  8374,
  1338,
  19139,
  10717,
  19083,
  2205,
  16679,
  2205,
  6491,
  5846,
  6210,
  12628,
  8148,
  6818,
  15018,
  11943,
  3351,
  10182,


In [14]:
sent_length = 8400
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ... 10625  1060 16644]
 [    0     0     0 ... 18480 18226  4561]
 [    0     0     0 ... 15667 18078  1674]
 ...
 [    0     0     0 ... 10545 12268 16605]
 [    0     0     0 ... 12412  8927 13199]
 [    0     0     0 ...  8453 11809 15708]]


In [16]:
model = Sequential()
model.add(Embedding(voc_size,300,input_length=sent_length))
model.compile('adam','mse')

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8400, 300)         6000000   
                                                                 
Total params: 6000000 (22.89 MB)
Trainable params: 6000000 (22.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
embedded_docs[0]

array([    0,     0,     0, ..., 10625,  1060, 16644], dtype=int32)

In [23]:
model.predict(embedded_docs[0])

263/263 [==============================] - 0s 1ms/step


array([[-0.01085564,  0.03331957, -0.01349492, ..., -0.04043126,
        -0.02954906, -0.02192828],
       [-0.01085564,  0.03331957, -0.01349492, ..., -0.04043126,
        -0.02954906, -0.02192828],
       [-0.01085564,  0.03331957, -0.01349492, ..., -0.04043126,
        -0.02954906, -0.02192828],
       ...,
       [ 0.01109711, -0.03324226,  0.01909789, ...,  0.0077757 ,
        -0.03934176, -0.01877302],
       [-0.0321205 , -0.03920565,  0.01876006, ...,  0.00186821,
        -0.02153843,  0.04358384],
       [-0.04644973, -0.03714694, -0.03169326, ...,  0.04681278,
        -0.00957202,  0.00249141]], dtype=float32)

In [26]:
embedded_docs.shape

(50000, 8400)